In [39]:
# Create training videos
import cv2
import numpy as np
from time import sleep
import glob
import os
import sys

NUM_FRAMES = 100
SEQ_LENGTH = 20
TAKES_PER = 3
MOODS = ['EXCITED', 'RELAXED']

In [1]:
def capture(num_frames, path='out.avi', countdown=0):
    for i in reversed(range(max(0, countdown))):
        i = i or 'GO!'
        print('Countdown:', i)
        sleep(1)

    # Create a VideoCapture object
    cap = cv2.VideoCapture(0)

    # Check if camera opened successfully
    if (cap.isOpened() == False): 
        print("Unable to read camera feed")

    # Default resolutions of the frame are obtained.The default resolutions are system dependent.
    # We convert the resolutions from float to integer.
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))

    # Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.
    out = cv2.VideoWriter(path, cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))

    print('Recording started')
    for i in range(num_frames):

        ret, frame = cap.read()

        if ret == True:     
            # Write the frame into the file 'output.avi'
            out.write(frame)


    # When everything done, release the video capture and video write objects
    cap.release()
    out.release()
    print('Recording stopped')
    
for take in range(TAKES_PER):
    for mood in MOODS:
        path = 'data/{}{}.avi'.format(mood, take)
        print('Get ready to act:', mood)
        capture(NUM_FRAMES, path=path, countdown=3)

Get ready to act: EXCITED
Countdown: 2
Countdown: 1
Countdown: GO!
Recording started
Recording stopped
Get ready to act: RELAXED
Countdown: 2
Countdown: 1
Countdown: GO!
Recording started
Recording stopped
Get ready to act: EXCITED
Countdown: 2
Countdown: 1
Countdown: GO!
Recording started
Recording stopped
Get ready to act: RELAXED
Countdown: 2
Countdown: 1
Countdown: GO!
Recording started
Recording stopped
Get ready to act: EXCITED
Countdown: 2
Countdown: 1
Countdown: GO!
Recording started
Recording stopped
Get ready to act: RELAXED
Countdown: 2
Countdown: 1
Countdown: GO!
Recording started
Recording stopped
Get ready to act: EXCITED
Countdown: 2
Countdown: 1
Countdown: GO!
Recording started
Recording stopped
Get ready to act: RELAXED
Countdown: 2
Countdown: 1
Countdown: GO!
Recording started
Recording stopped
Get ready to act: EXCITED
Countdown: 2
Countdown: 1
Countdown: GO!
Recording started


KeyboardInterrupt: 

In [26]:
ls data_img

EXCITED0-0.jpg     EXCITED2-1.jpg     RELAXED0-10.jpg    RELAXED2-11.jpg
EXCITED0-1.jpg     EXCITED2-10.jpg    RELAXED0-11.jpg    RELAXED2-12.jpg
EXCITED0-10.jpg    EXCITED2-11.jpg    RELAXED0-12.jpg    RELAXED2-13.jpg
EXCITED0-11.jpg    EXCITED2-12.jpg    RELAXED0-13.jpg    RELAXED2-14.jpg
EXCITED0-12.jpg    EXCITED2-13.jpg    RELAXED0-14.jpg    RELAXED2-15.jpg
EXCITED0-13.jpg    EXCITED2-14.jpg    RELAXED0-15.jpg    RELAXED2-16.jpg
EXCITED0-14.jpg    EXCITED2-15.jpg    RELAXED0-16.jpg    RELAXED2-17.jpg
EXCITED0-15.jpg    EXCITED2-16.jpg    RELAXED0-17.jpg    RELAXED2-18.jpg
EXCITED0-16.jpg    EXCITED2-17.jpg    RELAXED0-18.jpg    RELAXED2-19.jpg
EXCITED0-17.jpg    EXCITED2-18.jpg    RELAXED0-19.jpg    RELAXED2-2.jpg
EXCITED0-18.jpg    EXCITED2-19.jpg    RELAXED0-2.jpg     RELAXED2-20.jpg
EXCITED0-19.jpg    EXCITED2-2.jpg     RELAXED0-20.jpg    RELAXED2-21.jpg
EXCITED0-2.jpg     EXCITED2-20.jpg    RELAXED0-21.jpg    RELAXED2-22.jpg
EXCITED0-20.jpg    EXCITED2-21.jpg    RELAXED0-22.jp

In [43]:
# Create training data
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
import numpy as np

model = VGG16(weights='imagenet', include_top=False)
#model.summary()

def get_frames_as_vectors(video_path):
    vidcap = cv2.VideoCapture(path)
    success, frame = vidcap.read()
    while success:
        #img = image.load_img(frame, target_size=(224, 224))
        #img_data = image.img_to_array(img)
        img_data = cv2.resize(frame,(224,224))
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)
        vgg16_feature = model.predict(img_data)
        yield vgg16_feature
        success,frame = vidcap.read()
    vidcap.release()

_Xs, _ys = [], []
for mood in MOODS:
    print(mood)
    y = 1 if mood == 'EXCITED' else 0
    for video_path in glob.glob('data/{}*.avi'.format(mood)):
        print('processing', video_path)
        for i, X in enumerate(get_frames_as_vectors(video_path)):
            sys.stdout.write('- now processing frame {:03}   \r'.format(i))
            sys.stdout.flush()
            _Xs.append(X); _ys.append(y)



EXCITED
processing data_vid/EXCITED1.avi
processing data_vid/EXCITED0.avi
processing data_vid/EXCITED2.avi
processing data_vid/EXCITED3.avi
RELAXEDrocessing frame 099   
processing data_vid/RELAXED2.avi
processing data_vid/RELAXED3.avi
processing data_vid/RELAXED1.avi
processing data_vid/RELAXED0.avi


In [71]:
Xs = np.concatenate([np.expand_dims(a.flatten(), axis=0) for a in _Xs ])
ys = np.expand_dims(np.array(_ys), axis=1)
print('X/y dimensions:', Xs.shape, ys.shape)

X/y dimensions: (800, 25088) (800, 1)


(800, 25088)

In [58]:
np.expand_dims(np.array(range(10)), axis=1)

array([[0],
       [1],
       [2],
       [3],
       [4],
       [5],
       [6],
       [7],
       [8],
       [9]])